# spaCyチュートリアル
#### 02 spaCy中級

## モデルを読み込む

In [1]:
import spacy
import pandas as pd

In [2]:
# nlpオブジェクトを作成
nlp = spacy.load("ja_ginza")

## 単語ベクトル  
spaCyの英語モデルには小サイズを除いてGloVe（Word2Vecに似たアルゴリズム）で学習された単語ベクトルが付属されています。spaCyの単語ベクトルは他の学習済みモデルやGensim、FastTextやTensorFlowなどで一から学習した自作モデルに置き換える事も可能です。  

下記の例では単語や文同士の類似度を計算しています。単語や文のベクトル表現は文章分類などの下流タスクの特徴量としても使え、お好みの機械学習フレームワークに組み込むこともできます。  

GiNZAに付属されている単語ベクトルの次元数は100次元です。

In [3]:
# 猫の単語ベクトルと次元数をプリントする
単語1 = nlp("猫")
print(f"{単語1.vector} 次元数: {len(単語1.vector)}")

[-0.42628956 -0.25070515  0.56632304 -0.04601398 -0.5660679  -1.8086241
 -1.0238795   1.0143362   0.47763383 -0.26343933  1.5452907   1.0010606
  0.828702    1.0120491  -1.5566534  -0.53350675  0.08443125  1.9355137
 -0.00412369  0.613177    0.6531602   1.4621313  -0.41238445  0.5383796
  1.7463263   1.2742922   0.6872132   0.95741665  0.48112172  1.3087916
 -0.05922482 -1.0843756  -2.6484137  -0.30362067 -0.5912086   1.1504346
  1.8826097   0.77959156  1.2613478  -0.03555335 -1.2421137  -0.07365129
  0.6298954   0.65817195  1.3989674   1.3825057  -2.0864732  -0.8521881
  1.8979715  -0.57006633 -1.027578   -0.59627306  1.6408412  -0.6937744
 -0.80343324  0.57692754 -0.84792084 -0.3337947   1.0805885  -0.504921
 -0.31374794  0.6693753  -0.75144905  0.89448977 -1.3114382  -0.640819
 -0.47732472  0.36794803 -1.2829485   1.681444    0.32638294 -1.4287771
  0.9807352   0.35011157 -1.7619439   0.2487583  -1.767415    0.8563484
  1.4821373  -0.6031163   0.45007294  1.124629   -1.7681456   1.0

In [6]:
# 猫とネコのコサイン類似度
単語2 = nlp("ネコ")
print(単語1.similarity(単語2))

0.7506026693586216


In [7]:
# 文章全体のベクトルを平均することで2つの文章のコサイン類似度を計算する
文章1 = nlp("お寿司が大好きです。")
文章2 = nlp("好きな食べ物はラーメンです。")
print(文章1.similarity(文章2))

0.819816560369018


In [10]:
# 関係のない文章の類似度は低い
文章1 = nlp("お寿司が大好きです。")
文章2 = nlp("Jupyter Notebookのインストレーションガイド")
print(文章1.similarity(文章2))

0.07853908097411089


## 大規模データ処理   
このデータは[github.com/sinjorjob/django-transformer](https://github.com/sinjorjob/django-transformer)からダウンロードしました。

In [13]:
# !wget https://github.com/sinjorjob/django-transformer/raw/master/data/train.tsv -P data
# !wget https://github.com/sinjorjob/django-transformer/raw/master/data/test.tsv -P data

In [16]:
# !bzip2 data/train.tsv
# !bzip2 data/test.tsv

In [35]:
行列 = pd.read_csv('data/train.tsv.bz2', sep='\t', compression='bz2', names=['テキスト', 'ラベル'])
行列 = 行列.sample(1000, random_state=1111)
行列.shape

(1000, 2)

In [36]:
行列.head(10)

,テキスト,ラベル
1396,アルコール測定器の販売は好調に推移いたしました,1
603,当連結会計年度におけるわが国経済は、英国のEU離脱問題や米国の新大統領の誕生、また朝鮮半島情...,0
85,"事業の種類別では、設備工事業の受注高は1,342億94百万円（前期比4.5%減）、売上高は1...",0
1300,一方利益面では、営業利益が196億円、経常利益が197億56百万円、親会社株主に帰属する当期...,1
1928,① 「銀行・証券業務」におきましては、収益面では、有価証券利息配当金は増加したものの貸出金...,1
863,石炭部門につきましては、主力納入先である電力会社向け販売数量の増加により堅調に進みましたが、石油,1
1143,当セグメントの売上高は205億12百万円となり、前連結会計年度に比べ36百万円の増収となりました,1
1056,利益につきましては、増収を達成するなかで製造コストの効率化及び販売コストの効果的な運用に努め...,1
1804,一方、高圧分野におきましても、新たな需要家の獲得を進め、電力販売量を大きく拡大させました,1
1121,当期の売上高は、精密化学品事業部門が販売数量の増加と価格修正効果により増収となったため、46...,1


### 典型的なやり方 (低速)
シングルスレッド

In [37]:
def トークン化(テキスト:str=None):
    文章 = nlp(テキスト)
    トークンリスト = []
    
    for トークン in 文章:
        トークンリスト.append(トークン.text)
        
    return トークンリスト

In [39]:
%%timeit
行列['トークンリスト1'] = 行列.apply(lambda x: トークン化(x.テキスト), axis=1)

18.7 s ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
行列.head(10)

,テキスト,ラベル,トークンリスト1
1396,アルコール測定器の販売は好調に推移いたしました,1,"[アルコール, 測定, 器, の, 販売, は, 好調, に, 推移, いたし, まし, た]"
603,当連結会計年度におけるわが国経済は、英国のEU離脱問題や米国の新大統領の誕生、また朝鮮半島情...,0,"[当, 連結, 会計年度, に, おけ, る, わが, 国, 経済, は, 、, 英国, の..."
85,"事業の種類別では、設備工事業の受注高は1,342億94百万円（前期比4.5%減）、売上高は1...",0,"[事業, の, 種類, 別, で, は, 、, 設備, 工事, 業, の, 受注, 高, は..."
1300,一方利益面では、営業利益が196億円、経常利益が197億56百万円、親会社株主に帰属する当期...,1,"[一方, 利益, 面, で, は, 、, 営業, 利益, が, 196億, 円, 、, 経常..."
1928,① 「銀行・証券業務」におきましては、収益面では、有価証券利息配当金は増加したものの貸出金...,1,"[①, , 「, 銀行, ・, 証券, 業務, 」, に, おき, まし, ては, 、, ..."
863,石炭部門につきましては、主力納入先である電力会社向け販売数量の増加により堅調に進みましたが、石油,1,"[石炭, 部門, に, つき, まし, ては, 、, 主力, 納入先, で, ある, 電力,..."
1143,当セグメントの売上高は205億12百万円となり、前連結会計年度に比べ36百万円の増収となりました,1,"[当, セグメント, の, 売上高, は, 205億12百万, 円, と, なり, 、, 前..."
1056,利益につきましては、増収を達成するなかで製造コストの効率化及び販売コストの効果的な運用に努め...,1,"[利益, に, つき, まし, ては, 、, 増収, を, 達成, する, なか, で, 製..."
1804,一方、高圧分野におきましても、新たな需要家の獲得を進め、電力販売量を大きく拡大させました,1,"[一方, 、, 高圧, 分野, に, おき, まし, て, も, 、, 新た, な, 需要,..."
1121,当期の売上高は、精密化学品事業部門が販売数量の増加と価格修正効果により増収となったため、46...,1,"[当期, の, 売上高, は, 、, 精密, 化学品, 事業, 部門, が, 販売, 数量,..."


### spaCy独自のやり方 (高速)  
spaCyはpipeメソッドを使うことでテキストを並列処理してDocオブジェクトを返します。

In [41]:
%%timeit
トークンリスト = []

for 文章 in nlp.pipe(行列.テキスト.astype('unicode').values, batch_size=100, n_threads=40):
    単語リスト = []
    for トークン in 文章:
        単語リスト.append(トークン.text)
        
    トークンリスト.append(単語リスト)

行列['トークンリスト2'] = トークンリスト

16.1 s ± 379 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### spaCy独自のやり方 (超高速)  
`nlp`オブジェクトが作られる際にspaCyはパイプラインを追加します。不要なコンポーネントを無効にすることでspaCyの処理を更に高速化できます。パイプラインはカスタマイズ可能です。  

以下の図は英語版です。GiNZAではnerの後にJapaneseCorrectorで形態素解析誤り（トークン分割、品詞）の補修をしています。また、taggerが別のコンポーネントに組み込まれているようです。

![nlp_pipeline](img/nlp_pipeline.png)  

In [63]:
# パイプラインのコンポーネントをプリントする
for コンポーネント in nlp.pipeline:
    print(コンポーネント)

('parser', <spacy.pipeline.pipes.DependencyParser object at 0x7fb163042468>)
('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x7fb163042408>)
('JapaneseCorrector', <ginza.japanese_corrector.JapaneseCorrector object at 0x7fb1efbcd9b0>)


In [44]:
%%timeit
トークンリスト = []

# トークン化しか必要でない為、不要なコンポーネントは無効にする 
# nlp.make_docメソッドでも同様の処理が可能です。
with nlp.disable_pipes('parser', 'ner', 'JapaneseCorrector'):
    for 文章 in nlp.pipe(行列.テキスト.astype('unicode').values, batch_size=100, n_threads=40):
        単語リスト = []
        for トークン in 文章:
            単語リスト.append(トークン.text)

        トークンリスト.append(単語リスト)

行列['トークンリスト3'] = トークンリスト

11.3 s ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


ただGiNZA特有の問題で、JapaneseCorrectorがトークン分割の補修をすることがあるため、トークン化の精度は落ちます。そのため、`ner`だけ無効にすることで精度を犠牲にせず高速化できます。

In [61]:
行列['トークン分割補修あり'] = 行列.apply(lambda x: 0 if x.トークンリスト1 == x.トークンリスト3 else 1, axis=1)

In [54]:
# トークン分割の補修が8つの文章でされている
行列.トークン分割補修あり.value_counts()

0    992
1      8
Name: トークン分割補修あり, dtype: int64

In [52]:
# 補修後（左）と補修前（右）
行列[行列.トークン分割補修あり == 1][['トークンリスト1', 'トークンリスト3']]

,トークンリスト1,トークンリスト3
1377,"[ＨＤＤ, 用, サスペンション, は, 、, 既存, 製品, の, 需要, が, 回復, ...","[ＨＤＤ, 用, サスペンション, は, 、, 既存, 製, 品, の, 需要, が, 回復..."
1051,"[粉体, 調味料, 群, に, おい, て, は, 、, 小売, 用, 製品, で, は, ...","[粉体, 調味料, 群, に, おい, て, は, 、, 小売, 用, 製, 品, で, は..."
1302,"[工業製品, に, つい, て, は, 、, ふっ素, 樹脂, 製品, 、, 環境, 製, ...","[工業製品, に, つい, て, は, 、, ふっ素, 樹脂, 製, 品, 、, 環境, 製..."
1817,"[畜産, 品, は, 、, スーパーマーケット, や, 飲食店, 向け, 加工, 製品, の...","[畜産, 品, は, 、, スーパーマーケット, や, 飲食店, 向け, 加工, 製, 品,..."
1743,"[建築, ・, エクステリア, 部門, に, つき, まし, ては, 、, 新設, 住宅, ...","[建築, ・, エクステリア, 部門, に, つき, まし, ては, 、, 新設, 住宅, ..."
1047,"[液体, 調味料, 群, に, おい, て, は, 、, 小売, 用, 製品, で, は, ...","[液体, 調味料, 群, に, おい, て, は, 、, 小売, 用, 製, 品, で, は..."
1303,"[高機能, 製品, に, つい, て, は, 、, 半導体, ・, 液晶, 製造, 装置, ...","[高機能, 製, 品, に, つい, て, は, 、, 半導体, ・, 液晶, 製造, 装置..."
1229,"[道路, 用, 塗料, に, おい, て, は, 、, 補修, 関連, 製品, と, 視覚,...","[道路, 用, 塗料, に, おい, て, は, 、, 補修, 関連, 製, 品, と, 視..."
